## Bulk functional analysis for Control = 10mix and perturbation = 11mix

### Import libraries

In [ ]:
import decoupler as dc

# Only needed for visualization:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from anndata import AnnData

### Read data from  DESe2

In [ ]:
# Differencial expression analysis
path_11mix_vs_10mix = "/data/projects/2021/MicrobialMetabolites/bacterial-supernatant/20_deseq2icbi/paired_grp/deseq2_11mix_vs_10mix/11mix_10mix_IHWallGenes.tsv"

IHWallGenes = pd.read_csv(path_11mix_vs_10mix, sep="\t")

#### Remove NaN from Log2FC and pvalue columns

In [ ]:
IHWallGenes["log2FoldChange"] = IHWallGenes["log2FoldChange"].fillna(value=0)
IHWallGenes["pvalue"] = IHWallGenes["pvalue"].fillna(value=1)

In [ ]:
IHWallGenes["log2FoldChange"].min()

In [ ]:
IHWallGenes["log2FoldChange"].max()

In [ ]:
IHWallGenes["pvalue"].min()

In [ ]:
IHWallGenes["pvalue"].max()

#### Create the logFCs_icbi and pvalue_icbi dataframes

In [ ]:
logFCs_icbi = IHWallGenes[["log2FoldChange", "gene_name"]].set_index("gene_name").T
logFCs_icbi.index = ("stat",)

In [ ]:
logFCs_icbi

In [ ]:
pvalue_icbi = IHWallGenes[["pvalue", "gene_name"]].set_index("gene_name").T
pvalue_icbi.index = ("stat",)
pvalue_icbi

In [ ]:
pvalue_icbi

In [ ]:
logFCs_icbi.shape

#### Set name to None to avoid mistakes with dc function 

In [ ]:
pvalue_icbi.columns.name = None
logFCs_icbi.columns.name = None

In [ ]:
logFCs_icbi

In [ ]:
pvalue_icbi

### Differential expression analysis 

In [ ]:
dc.plot_volcano(logFCs_icbi, pvalue_icbi, "stat", top=15, sign_thr=0.05, lFCs_thr=0.5)

To excract the significant genes after FDR correction (p adjusted ). We have padjusted in the table IHWallGenes.tsv

In [ ]:
top_genes = dc.get_top_targets(
    logFCs_icbi,
    pvalue_icbi,
    name="deseq2_estimate",
    contrast="stat",
    sign_thr=0.05,
    lFCs_thr=0.5,
);

In [ ]:
top_genes

###  Pathwax activity inference

To estimate pathway actrivity. (upregulated and downregulted) Activities and p values for each pathway.

In [ ]:
# Retrieve PROGENy model weights
progeny = dc.get_progeny(organism="mouse", top=300)

# Infer pathway activities with consensus
pathway_acts_icbi, pathway_pvals_icbi = dc.run_consensus(mat=logFCs_icbi, net=progeny)
pathway_acts_icbi

In [ ]:
pathway_pvals_icbi

### P value correction

- Thus, to calculate the Benjamini-Hochberg critical value for each p-value,
- we can use the following formula: (i/20)*0.2 where i = rank of p-value
- qvalue = An FDR-adjusted p-value

In [ ]:
pathway_pvals_icbi.columns

In [ ]:
df = pathway_pvals_icbi.T
col_one_list = df["stat"].tolist()

In [ ]:
from statsmodels.stats.multitest import fdrcorrection

rejected, qvalue = fdrcorrection(col_one_list)

In [ ]:
qvalue

In [ ]:
pathway_pvals_icbi.columns

In [ ]:
df_qvalues = pd.DataFrame(index = pathway_pvals_icbi.columns, data = qvalue)
qvalues_pathway =df_qvalues.sort_values(0).T


In [ ]:
dc.plot_barplot(pathway_acts_icbi, "stat", top=25, vertical=False)

In [ ]:
dc.plot_volcano(pathway_acts_icbi, pathway_pvals_icbi, "stat")

In [ ]:
qvalues_pathway.columns.name = None

In [ ]:
qdf_pw = qvalues_pathway.T
qdf_pw .rename(columns ={0:"stat"}, inplace = True)
qdf_pw 

In [ ]:
qvalues_pathway = qdf_pw.T

In [ ]:
qvalues_pathway

Visualize the most active and inactive transcription factors

In [ ]:
dc.plot_volcano(pathway_acts_icbi,qvalues_pathway, "stat")

Visualize  the pathways that increased or decreased their activty 

Visualize the targets of NFkB and JAK-STAT
* Below the data is treated to get rid of duplicated gene names* 

In [ ]:
trans = logFCs_icbi.T
trans["gene_name"] = trans.index

In [ ]:
trans

In [ ]:
name_counts = trans["gene_name"].value_counts()

In [ ]:
trans.loc[name_counts[name_counts == 1].index]

In [ ]:
name_counts

In [ ]:
name_counts[name_counts == 1]

In [ ]:
name_counts[name_counts == 1].index

In [ ]:
logFCs_icbi_nodups = logFCs_icbi.loc[:,  name_counts[name_counts == 1].index]

In [ ]:
trans_p = pvalue_icbi.T
trans_p["gene_name"] = trans_p.index

In [ ]:
name_counts_p = trans_p["gene_name"].value_counts()

In [ ]:
name_counts_p

In [ ]:
trans_p.loc[name_counts_p[name_counts_p==1].index]

In [ ]:
name_counts_p[name_counts_p==1]

In [ ]:
pvalue_icbi_nodups = pvalue_icbi.loc[:,  name_counts_p[name_counts_p == 1].index]


In [ ]:
# Volcano plot for the targets of NFkB

dc.plot_volcano(
    logFCs_icbi_nodups,
    pvalue_icbi_nodups,
    "stat",
    name="NFkB",
    net=progeny,
    top=5,
    sign_thr=0.05,
    lFCs_thr=0.5,
)

In [ ]:
# Volcano plot for the targets of JAK-STAT

dc.plot_volcano(
    logFCs_icbi_nodups,
    pvalue_icbi_nodups,
    "stat",
    name="JAK-STAT",
    net=progeny,
    top=5,
    sign_thr=0.05,
    lFCs_thr=0.5,
)

### Transcription factor activity

Estimate transcription factor activity. We obtain p values and activities for each transcription factor. 

In [ ]:
# Retrieve DoRothEA gene regulatory network
dorothea = dc.get_dorothea(organism="mouse")

# Infer pathway activities with consensus
tf_acts_icbi, tf_pvals = dc.run_consensus(mat=logFCs_icbi, net=dorothea)
tf_acts_icbi

In [ ]:
tf_pvals

### P value correction

- Thus, to calculate the Benjamini-Hochberg critical value for each p-value,
- we can use the following formula: (i/20)*0.2 where i = rank of p-value
- qvalue = An FDR-adjusted p-value

In [ ]:
tf_pvals.columns

In [ ]:
df2 = tf_pvals.T
col_one_list2 = df2["stat"].tolist()

In [ ]:
from statsmodels.stats.multitest import fdrcorrection

rejected, qvalue2 = fdrcorrection(col_one_list2)

In [ ]:
qvalue2

In [ ]:
tf_pvals.columns

In [ ]:
df_qvalues2 = pd.DataFrame(index = tf_pvals.columns, data = qvalue2)
df_qvalues2.sort_values(0).T


In [ ]:
df_qvalues2.rename(columns ={0:"stat"}, inplace = True)

In [ ]:
qvalues_tf = df_qvalues2.T

In [ ]:
qvalues_tf

In [ ]:
tf_pvals

In [ ]:
qvalues_tf .columns.name = None

Visualize the most active and inactive transcription factors

In [ ]:
dc.plot_volcano(tf_acts_icbi,qvalues_tf, "stat")

In [ ]:
dc.plot_volcano(tf_acts_icbi, tf_pvals, "stat")

In [ ]:
dc.plot_barplot(tf_acts_icbi, "stat", top=25, vertical=True)

* The data for the volcano plots is treated to get rid of duplicated gene names* 

In [ ]:
# Volcano plot to inpect the downstream targets of transcription factor E2f4 and Ahr

dc.plot_volcano(logFCs_icbi_nodups, pvalue_icbi_nodups, 'stat', name='E2f4', net=dorothea, top=5, sign_thr=0.05, lFCs_thr=0.5)

dc.plot_volcano(logFCs_icbi_nodups, pvalue_icbi_nodups, 'stat', name='Ahr', net=dorothea, top=5, sign_thr=0.05, lFCs_thr=0.5)

### Functional enrichment of biological terms

In [ ]:
# Retrieve MSigDB resource
msigdb = dc.get_resource('MSigDB')
msigdb


In [ ]:
# Filter by a desired geneset collection, for example hallmarks
msigdb = msigdb[msigdb['collection']=='hallmark']
msigdb = msigdb.drop_duplicates(['geneset', 'genesymbol'])

# Infer enrichment with ora using significant deg
top_genes = dc.get_top_targets(logFCs_icbi,pvalue_icbi, 'stat', sign_thr=0.05, lFCs_thr=1.5)
enr_pvals = dc.get_ora_df(top_genes, msigdb, groupby='contrast', features='name', source='geneset', target='genesymbol')
enr_pvals

In [ ]:
msigdb = msigdb[msigdb['collection']=='hallmark']

In [ ]:
msigdb

In [ ]:
logFCs_icbi.T